https://render.githubusercontent.com/view/ipynb?commit=0a25604af68824dc0def18dc9c64fd6e3316c68d&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f7370726f2f70726163746963616c2d7079746f7263682f306132353630346166363838323464633064656631386463396336346664366533333136633638642f636861722d726e6e2d636c617373696669636174696f6e2f636861722d726e6e2d636c617373696669636174696f6e2e6970796e62&nwo=spro%2Fpractical-pytorch&path=char-rnn-classification%2Fchar-rnn-classification.ipynb&repository_id=79684696&repository_type=Repository#Creating-the-Network

In [1]:
# Author: Ellis Brown
# http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

import random
import string
import math
import csv
import unicodedata
import nltk
import time
from nltk.corpus import names
from nltk.classify import apply_features
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
dataset_filename = "nltk_mbejda.csv"

## Helper Functions

In [3]:
def load_names():
    names = []
    genders = []
 
    with open(dataset_filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        for row in csvReader:
            names.append(row[0])
            genders.append(row[1])

    return names, genders

In [4]:
def load_dataset(shuffled=True):
    """Returns the name->gender dataset ready for processing
    
    Args:
        shuffled (Boolean, optional): set to False to return the dataset unshuffled
    Return:
        namelist (list(String,String)): list of (name, gender) records
    """
    nms,gns = load_names()
    namelist = list(zip(nms,gns))
    if shuffled:
        random.shuffle(namelist)
    return namelist

In [5]:
name_gender = {}
gender_name = {}
for name, gender in load_dataset():
    name_gender[name] = gender
    gender_name.setdefault(gender, []).append(name)

In [6]:
def clean_str(s):
    uncoded = ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in string.ascii_letters
    )
    return uncoded.lower()

In [7]:
all_letters = string.ascii_letters + " .,;'"
all_genders = ["male", "female"]
n_letters = len(all_letters)
n_genders = len(all_genders)

In [8]:
def name_to_tensor(name):
    """converts a name to a vectorized numerical input for use with a nn
    
    Args:
        name (string): full name (e.g., "Ellis Brown")
    
    Return:
        TODO
    """
    
    name = clean_str(name)
    tensor = torch.zeros(len(name), 1, n_letters)
    for li, letter in enumerate(name):
        letter_index = all_letters.find(letter)
        tensor[li][0][letter_index] = 1
    return tensor

In [9]:
def gender_from_output(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    gender_i = top_i[0][0]
    return all_genders[gender_i], gender_i

In [10]:
def random_training_pair():                                                                                                               
    gender = random.choice(all_genders)
    name = random.choice(gender_name[gender])
    gender_tensor = Variable(torch.LongTensor([all_genders.index(gender)]))
    name_tensor = Variable(name_to_tensor(name))
    return gender, name, gender_tensor, name_tensor

In [11]:
# "ASDffasdf".lower()[:5]
# list(map( lambda x: ord(x), "hello"))
ct = torch.CharTensor(5)
ct


-120
-101
  67
  -2
 112
[torch.CharTensor of size 5]

In [12]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax()
    
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

## Train

In [13]:
n_hidden = 128
n_epochs = 100000
print_every = 5000
plot_every = 1000
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

In [14]:
rnn = RNN(n_letters, n_hidden, n_genders)
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

In [15]:
def train(gender_tensor, name_tensor):
    hidden = rnn.init_hidden()
    optimizer.zero_grad()
    
    for i in range(name_tensor.size()[0]):
        output, hidden = rnn(name_tensor[i], hidden)

    loss = criterion(output, gender_tensor)
    loss.backward()

    optimizer.step()

    return output, loss.data[0]

In [16]:
# Keep track of losses for plotting
current_loss = 0
all_losses = []

def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [17]:
start = time.time()

print("Beginning training...")
for epoch in range(1, n_epochs + 1):
    gender, name, gender_tensor, name_tensor = random_training_pair()
    output, loss = train(gender_tensor, name_tensor)
    current_loss += loss

    # Print epoch number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = gender_from_output(output)
        correct = '✓' if guess == gender else '✗ (%s)' % gender
        print('%d %d%% (%s) %.4f %s / %s %s' % (epoch, epoch / n_epochs * 100, time_since(start), loss, name, guess, correct))

    # Add current loss avg to list of losses
    if epoch % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

Beginning training...


KeyboardInterrupt: 

In [ ]:
# torch.save(rnn, 'gender-rnn-classification.pt')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(all_losses)

## Evaluate

In [18]:
rnn = torch.load('gender-rnn-classification2.pt')

In [19]:
def evaluate(name_tensor):
    hidden = rnn.init_hidden()
    
    for i in range(name_tensor.size()[0]):
        output, hidden = rnn(name_tensor[i], hidden)
    
    return output

## Predict

In [20]:
def predict(name, n_predictions=2):
    output = evaluate(Variable(name_to_tensor(name)))

    # Get top N categories
    topv, topi = output.data.topk(n_predictions, 1, True)
    predictions = []

    for i in range(n_predictions):
        value = topv[0][i]
        gender_index = topi[0][i]
#         print('(%.2f) %s' % (value, all_genders[gender_index]))
        predictions.append([value, all_genders[gender_index]])

    return predictions

In [21]:
testset = load_dataset()[:1000]

In [22]:
p = predict("ellis", 1)
p[0][1]

'female'

In [23]:
def test(dset):
    cum = 0
    for name, gender in dset:
        prediction = predict(name,1)[0][1]
        if prediction == gender:
            cum += 1
    print("Accuracy: ", cum / len(dset))
        

In [24]:
# test(testset)

In [25]:
test(load_dataset())

Accuracy:  0.806583427922815


In [26]:
predict("arjun")

[[-0.2049436867237091, 'male'], [-1.6857423782348633, 'female']]